<a href="https://colab.research.google.com/github/balawillgetyou/dy/blob/master/spaCyTextClassifier20200210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective:
To demonstrate a Hierarchical Attention Network text classifer, built using spaCy. The data is from a hackathon where the author needs to be identified, from a paragraph of text. spaCy's default hyper parameter settings work well. To illustrate tuning these, a decaying drop out is also demonstrated and proves the point that hyper parameter tuning is extremely hard.  
The network is trained for just 5 iterations for this demo.

In [0]:
#library needed for text classifier training
import spacy
from spacy.util import minibatch, compounding, decaying

#library to load and process data
import os
import copy
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1) #to prevent cell display truncation

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from __future__ import unicode_literals
from operator import itemgetter


#to save trained model
#from pathlib import Path

#metrics calculation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.utils import resample#downsample not-an-obligation

Instructions to load data from Google Drive. Use the "Files" hamburger icon on the top left. Mount Drive. Ensure "drive" is visible in the folders icon.

In [0]:
def clean_string(mystring):
    return re.sub('[^A-Za-z\ 0-9 ]+', '', mystring)

In [0]:
def loadPreProcData():
    
    temp_1 = open("/content/drive/My Drive/Colab_Notebooks_Bala/WhoseLineIsItAnywayTRAIN.csv", 'r', encoding='latin-1') 
    WhoseLineData = pd.read_csv(temp_1)
    WhoseLineData = shuffle(WhoseLineData)
    WhoseLineData = WhoseLineData.iloc[0:4999,]

    #data split to execute validations using unseen data
    df_train, df_val = train_test_split(WhoseLineData, test_size=0.15)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    #df_val.to_csv("/content/drive/My Drive/Colab_Notebooks_Bala/trainedModel/WhoseLine_val.csv")

    #conversion from dataframe to a list of nested dictionaries that look like this:
    #('The grey donkey paid no attention to thisspeech because he had just stopped before a house which had painted overthe doorway a pair of hoofs with a donkey tail between them and a rudecrown and sceptre aboveIll see if his magnificent Majesty King Kikabray is at home saidhe He lifted his head and called Wheehaw wheehaw wheehaw threetimes in a shocking voice turning about and kicking with his heelsagainst the panel of the door For a time there was no reply then thedoor opened far enough to permit a donkeys head to stick out and lookat themIt was a white head with big awful ears and round solemn eyesHave the foxes gone it asked in a trembling voiceThey havent been here most stupendous Majesty replied the grey oneThe new arrivals prove to be travelers of distinctionOh said the King in a relieved tone of voice Let them come inHe opened the door wide and the party marched into a big room whichDorothy thought looked quite unlike a kings palace There were mats ofwoven grasses on the floor and the place was clean and neat but hisMajesty had no other furniture at allperhaps because he didnt needit He squatted down in the center of the room and a little brown donkeyran and brought a big gold crown which it placed on the monarchs headand a golden staff with a jeweled ball at the end of it which the Kingheld between his front hoofs as he sat uprightNow then said his Majesty waving his long ears gently to and frotell me why you are here and what you expect me to do for you Heeyed ButtonBright rather sharply as if afraid of the little boysqueer head though it was the shaggy man who undertook to replyMost noble and supreme ruler of Dunkiton he said trying not to laughin the solemn Kings face we are strangers traveling through yourdominions and have entered your magnificent city because the road ledthrough it and there was no way to go around All we desire is to payour respects to your Majestythe cleverest king in all the world Imsureand then to continue on our wayThis polite speech pleased the King very much indeed it pleased him somuch that it proved an unlucky speech for the shaggy man Perhaps theLove Magnet helped to win his Majestys affection as well as theflattery but however this may be the white donkey looked kindly uponthe speaker and saidOnly a donkey should be able to use such fine big words and you aretoo wise and admirable in all ways to be a mere man Also I feel that Ilove you as well as I do my own favored people so I will bestow uponyou the greatest gift within my powera donkeys headAs he spoke he waved his jeweled staff Although the shaggy man criedout and tried to leap backward and escape it proved of no use Suddenlyhis own head was gone and a donkey head appeared in its placea brownshaggy head so absurd and droll that Dorothy and Polly both broke intomerry laughter and even ButtonBrights fox face wore a smileDear me dear me cried the shaggy man feeling of his shaggy new headand his long ears What a misfortunewhat a great misfortune Give meback my own head you stupid kingif you love me at allDont you like it asked the King surprisedHeehaw I hate it Take it awayquick said the shaggy manKING KICKABRAY WORKS MAGIC ON THE SHAGGY MANBut I cant do that was the reply My magic works only one way',
    #{'cats': {0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}})
    
    author = WhoseLineData['author'].unique()
    labels_default = dict((v, 0) for v in author)

    train_data = []
    for i, row in df_train.iterrows():

        label_values = copy.deepcopy(labels_default)
        label_values[row['author']] = 1
        train_data.append((str(clean_string(row['text'])), {"cats": label_values}))
    
    return train_data, df_val

In [0]:
#this function uses default spaCy hyperparameter settings, including the optimizer, minibatch size and drop out
def trainSpacy(model=None, output_dir=None, n_iter=5):
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'textcat' not in nlp.pipe_names:
        textcat = nlp.create_pipe('textcat')
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe('textcat')

    # add label to text classifier
    for i in [0,1,2,3,4,5,6,7,8,9]:
        textcat.add_label(i)

    train_data, df_val = loadPreProcData()

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        print("Training the model...")
        print('{:^5}\t'.format('LOSS'))
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.25,losses=losses)
            
            print('{0:.3f}'  # print a simple table
                  .format(losses['textcat']))
    return [nlp, df_val]

In [0]:
#this function demonstrates drop out decay
def trainSpacyDropOut(model=None, output_dir=None, n_iter=5):
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'textcat' not in nlp.pipe_names:
        textcat = nlp.create_pipe('textcat')
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe('textcat')

    # add label to text classifier
    for i in [0,1,2,3,4,5,6,7,8,9]:
        textcat.add_label(i)

    train_data, df_val = loadPreProcData()

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        print("Training the model...")
        print('{:^5}\t'.format('LOSS'))
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            dropout = decaying(0.6, 0.2, 1e-4)
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=next(dropout),losses=losses)
            
            print('{0:.3f}'  # print a simple table
                  .format(losses['textcat']))
    return [nlp, df_val]

In [0]:
def predictObligationType(nlp, df):
    predictedClass = []
    texts = df['text']
    
    for aa in range(len(df)):
        test_text = texts.iloc[aa]
        doc = nlp(test_text)
        lis = doc.cats.items()
        predictedClass.append(max(lis, key=itemgetter(1))[0])
        
    df['Prediction'] = pd.DataFrame(predictedClass)

 
    return(df)

In [0]:
def scoreIt(nlp, df):

    predictions = predictObligationType(nlp, df)
    
    actualClass = df['author']
    predictedClass = df['Prediction']

    precision, recall, fscore, support = score(actualClass, predictedClass, labels = [0,1,2,3,4,5,6,7,8,9])

    results1 = pd.concat((pd.Series([0,1,2,3,4,5,6,7,8,9]), pd.Series(precision), pd.Series(recall), pd.Series(fscore), pd.Series(support)), axis=1, join = 'outer')
    results1.columns=('label','precision', 'recall', 'fscore', 'support')
    print('spaCy','\n','*'*50,'\n',round(results1,2))

First we train the classifier with default settings

In [57]:
nlp, df_val = trainSpacy()

Created blank 'en' model
Training the model...
LOSS 	
26.309
13.997
8.423
5.459
3.408


In [58]:
scoreIt(nlp, df_val)

spaCy 
 ************************************************** 
    label  precision  recall  fscore  support
0  0      0.48       0.91    0.63    133    
1  1      0.52       0.68    0.59    22     
2  2      1.00       0.72    0.84    118    
3  3      0.96       0.92    0.94    51     
4  4      0.73       0.98    0.83    128    
5  5      0.86       0.31    0.45    144    
6  6      0.72       0.76    0.74    37     
7  7      0.97       0.67    0.79    42     
8  8      0.86       0.71    0.78    35     
9  9      1.00       0.35    0.52    40     


Next we train the classifier with updated hyper parameters. Hard to see any improvement.

In [62]:
nlpDropOut, df_valDropOut = trainSpacyDropOut()
scoreIt(nlpDropOut, df_valDropOut)

Created blank 'en' model
Training the model...
LOSS 	
35.087
21.581
16.405
13.397
11.527
spaCy 
 ************************************************** 
    label  precision  recall  fscore  support
0  0      0.98       0.76    0.85    135    
1  1      0.63       0.57    0.60    30     
2  2      0.50       0.95    0.66    107    
3  3      0.65       0.96    0.78    53     
4  4      0.64       0.92    0.75    145    
5  5      0.97       0.27    0.42    131    
6  6      0.33       0.07    0.11    30     
7  7      0.84       0.55    0.67    47     
8  8      0.00       0.00    0.00    32     
9  9      0.23       0.32    0.27    40     


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
